**Importing the libraries**

In [1]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
tf.__version__

'2.9.1'

**Data Preprocessing**

--split folders

In [2]:
import splitfolders
splitfolders.ratio("alphabet letters", output="Alphabet  dataset", ratio = (.7,.2,.1), group_prefix=None, move = True)

--Preprocessing the Training set
- Data Augmantation 
- import data

In [2]:
train_datagen = ImageDataGenerator(
                                    #width_shift_range = 0.3,
                                    #height_shift_range = 0.3,
                                    brightness_range = (0.2, 1.4), 
                                    rescale = 1./255,
                                    zoom_range = [0.5, 1.5],
                                    channel_shift_range=150.0,
                                    horizontal_flip = True
                                   )
training_set = train_datagen.flow_from_directory('alphabet demo dataset\\train',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')



Found 26250 images belonging to 25 classes.


Preprocessing the Test set
- import validaiton data

In [3]:

val_datagen = ImageDataGenerator(rescale = 1./255)
val_set = val_datagen.flow_from_directory('alphabet demo dataset\\val',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 7500 images belonging to 25 classes.


**Building the CNN**

--Initialising the CNN

In [4]:
cnn = tf.keras.models.Sequential()

Step 1 - Convolution

In [5]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

Step 2 - Pooling

In [6]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

Adding a second convolutional layer

In [7]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

Step 3 - Flattening

In [8]:
cnn.add(tf.keras.layers.Flatten())

Step 4 - Full Connection

In [9]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

Step 5 - Output Layer

In [10]:
cnn.add(tf.keras.layers.Dense(units=25, activation='softmax'))

**Training the CNN**

--Compiling the CNN

In [11]:
cnn.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

Training the CNN on the Training set and evaluating it on the Test set

In [12]:
cnn.fit(x = training_set, validation_data = val_set, epochs = 30)

Epoch 1/30
821/821 [==============================] - 287s 341ms/step - loss: 2.5379 - accuracy: 0.2740 - val_loss: 1.4250 - val_accuracy: 0.5817
Epoch 2/30
821/821 [==============================] - 251s 306ms/step - loss: 1.8801 - accuracy: 0.4531 - val_loss: 0.9554 - val_accuracy: 0.7167
Epoch 3/30
821/821 [==============================] - 203s 247ms/step - loss: 1.6230 - accuracy: 0.5265 - val_loss: 0.7503 - val_accuracy: 0.7685
Epoch 4/30
821/821 [==============================] - 217s 265ms/step - loss: 1.4354 - accuracy: 0.5763 - val_loss: 0.6823 - val_accuracy: 0.7904
Epoch 5/30
821/821 [==============================] - 199s 243ms/step - loss: 1.3516 - accuracy: 0.6017 - val_loss: 0.5886 - val_accuracy: 0.8183
Epoch 6/30
821/821 [==============================] - 204s 248ms/step - loss: 1.2584 - accuracy: 0.6275 - val_loss: 0.5440 - val_accuracy: 0.8333
Epoch 7/30
821/821 [==============================] - 199s 242ms/step - loss: 1.1986 - accuracy: 0.6462 - val_loss: 0.5151 -

Part 4 - Making a single prediction

In [13]:
training_set.class_indices

{'a': 0,
 'b': 1,
 'c': 2,
 'd': 3,
 'f': 4,
 'g': 5,
 'h': 6,
 'i': 7,
 'j': 8,
 'k': 9,
 'l': 10,
 'm': 11,
 'n': 12,
 'o': 13,
 'p': 14,
 'q': 15,
 'r': 16,
 's': 17,
 't': 18,
 'u': 19,
 'v': 20,
 'w': 21,
 'x': 22,
 'y': 23,
 'z': 24}

In [14]:
import numpy as np
from keras.preprocessing import image
test_image = tf.keras.utils.load_img('Alphabet  dataset\\test\\b\\b_4.jpg', target_size = (64, 64))
test_image = tf.keras.utils.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
print(result)

1/1 [==============================] - 5s 5s/step
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]]


**Create model file**

In [15]:
cnn.save('demo_cnn_model_version_3.h5')

--load and sum the model

In [16]:
model = tf.keras.models.load_model('demo_cnn_model_version_3.h5')

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 128)               8

**One image tests**


predict

In [18]:

import numpy as np
test_image = tf.keras.utils.load_img('Alphabet  dataset\\test\\b\\b_4.jpg', target_size = (64, 64))
test_image = tf.keras.utils.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
print(test_image)
result = model.predict(test_image)
print(result)


[[[[121. 121. 121.]
   [114. 114. 114.]
   [154. 154. 154.]
   ...
   [247. 247. 247.]
   [246. 246. 246.]
   [244. 244. 244.]]

  [[156. 156. 156.]
   [114. 114. 114.]
   [ 78.  78.  78.]
   ...
   [252. 252. 252.]
   [251. 251. 251.]
   [250. 250. 250.]]

  [[151. 151. 151.]
   [165. 165. 165.]
   [163. 163. 163.]
   ...
   [253. 253. 253.]
   [252. 252. 252.]
   [251. 251. 251.]]

  ...

  [[  2.   2.   2.]
   [  2.   2.   2.]
   [  2.   2.   2.]
   ...
   [ 37.  37.  37.]
   [ 32.  32.  32.]
   [ 29.  29.  29.]]

  [[  2.   2.   2.]
   [  2.   2.   2.]
   [  2.   2.   2.]
   ...
   [ 38.  38.  38.]
   [ 36.  36.  36.]
   [ 34.  34.  34.]]

  [[  2.   2.   2.]
   [  2.   2.   2.]
   [  2.   2.   2.]
   ...
   [ 27.  27.  27.]
   [ 28.  28.  28.]
   [ 29.  29.  29.]]]]
1/1 [==============================] - 1s 561ms/step
[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]]


final result

In [19]:
letter_position = 0
for i in result[0]:   
    if i != 0:
        print(i)
        position = list(training_set.class_indices.values()).index(letter_position)
        print(position)
        print(list(training_set.class_indices.keys())[position])
    letter_position += 1

1.0
1
b
